## Train NER Model using Pytorch

We will use Flair which is a pytorch based popular NLP framework to train the model. This is a custom library which can be replaced by any other library. 

### Upload the training files to S3

In [8]:
import sagemaker
import time
import boto3

In [9]:
sagemaker_session = sagemaker.Session()

In [12]:
inputs = sagemaker_session.upload_data(path='data/', key_prefix='data/ner-dataset')

In [13]:
print("Training files uploaded here : {}".format(inputs))

Training files uploaded here : s3://sagemaker-us-east-1-275443674968/data/ner-dataset


### Train model using Sagemaker Pytorch Estimator

In [14]:
import sys
import IPython
install_needed = False  # should only be True once
if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip install -U smdebug
    IPython.Application.instance().kernel.do_shutdown(True)

In [15]:
from sagemaker.pytorch import PyTorch
role = sagemaker.get_execution_role()

In [35]:
estimator = PyTorch(entry_point='train.py',
                    source_dir='code',
                    role=role,
                    framework_version='1.4.0',
                    py_version="py3",
                    train_instance_count=1,
                    train_instance_type='ml.g5.xlarge',
                    hyperparameters={
                        'epochs': 25,
                        'learning_rate': 0.1,
                        'hidden_size':128},
                    train_use_spot_instances=False)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [36]:
estimator.fit({'train': inputs + "/train.txt","validation":inputs + "/train.txt","eval":inputs  + "/train.txt"})

#estimator.fit({'train': inputs,"validation":inputs,"eval":inputs})


2022-06-28 17:28:17 Starting - Starting the training job...
2022-06-28 17:28:34 Starting - Preparing the instances for trainingProfilerReport-1656437297: InProgress
.........
2022-06-28 17:30:16 Downloading - Downloading input data......
2022-06-28 17:31:17 Training - Downloading the training image.......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-06-28 17:32:26,213 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2022-06-28 17:32:26,239 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-06-28 17:32:26,243 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-06-28 17:32:26,552 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2022-06-28 17:32:26,552 sagemaker-containers INFO     Generating setup.cfg
2022-06-28 17:32:26,552 sagemaker-containers INF

### Deploy Model using Sagemaker Model class.

1. Download the output files from the training job
2. Archieve inference code and dependencies along with the model object.
3. Upload the archieve to s3
4. Create a pytorch Model and deploy.
5. Invoke the model
6. Cleanup

#### 1. Download and unzip output files from training job

In [37]:
model_path = estimator.latest_training_job.describe()["ModelArtifacts"]["S3ModelArtifacts"]

In [38]:
bucket_name = model_path.replace("s3://","").split("/")[0]
prefix = "/".join(model_path.replace("s3://","").split("/")[1:])

print("bucket name is {}".format(bucket_name))
print("File path is {}".format(prefix))

sagemaker_session.download_data(path="output/",bucket=bucket_name,key_prefix=prefix)

bucket name is sagemaker-us-east-1-275443674968
File path is pytorch-training-2022-06-28-17-28-17-450/output/model.tar.gz


In [39]:
import tarfile
model_tar = tarfile.open('output/model.tar.gz')
model_tar.extractall('output/') # specify which folder to extract to
model_tar.close()

#### 2. Create an tar containing the model file and inference code

We need to create a tar.gz file in the below format 

model.tar.gz/
    
        |- model.pt
    
        |- code/
    
          |- inference.py
      
          |- requirements.txt

In [40]:
# Create the required folder structure and copy the necessary contents

! mkdir deployment
! cp output/final-model.pt deployment/model.pt
! mkdir deployment/code
! cp code/inference.py deployment/code/inference.py
! cp code/requirements.txt deployment/code/requirements.txt

In [41]:
# Create the tar file 

import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('deployment',arcname='.')
archive.close()

#### 3. Upload Archieve/tar file to s3 

In [42]:
model_path = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='data/ner-dataset/model')

In [43]:
print("Model stored in this location {}".format(model_path))

Model stored in this location s3://sagemaker-us-east-1-275443674968/data/ner-dataset/model/model.tar.gz


#### 4. Create a pytorch Model and Deploy

In [44]:
from sagemaker.pytorch import PyTorchModel

In [46]:
from sagemaker import get_execution_role
role = get_execution_role()

pytorch_model = PyTorchModel(model_data=model_path, role=role,
                             entry_point='inference.py', framework_version='1.4.0',
                              py_version="py3", source_dir='code')

In [47]:
predictor = pytorch_model.deploy(instance_type='ml.t2.large', initial_instance_count=1)

----------!

#### 5. Invoke Model endpoint

In [48]:
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

response = predictor.predict({"text":"Class A CDL Truck Driver"})

print(response)

{'entities': [{'text': 'Class A CDL Truck Driver', 'start_pos': 0, 'end_pos': 24, 'label': 'job_role', 'confidence': 0.671200567483902}]}


In [49]:
predictor.delete_endpoint()